#Chatbot project using pytorch and NLTK3
### 1- load data:

In [1]:
from json import load
import Utils
with open("intents.json") as f:
    intents = load(f)['intents']

all_words = []
tags = []
xy = []

for intent in intents:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        words = Utils.tokenize(pattern)
        all_words.extend(words)
        xy.append((words,tag))


print('all words: \n',all_words)
print('tags: \n',tags)
print('xy: \n',xy)



all words: 
 ['Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Thank', "'s", 'a', 'lot', '!', 'Which', 'items', 'do', 'you', 'have', '?', 'What', 'kinds', 'of', 'items', 'are', 'there', '?', 'What', 'do', 'you', 'sell', '?', 'Do', 'you', 'take', 'credit', 'cards', '?', 'Do', 'you', 'accept', 'Mastercard', '?', 'Can', 'I', 'pay', 'with', 'Paypal', '?', 'Are', 'you', 'cash', 'only', '?', 'How', 'long', 'does', 'delivery', 'take', '?', 'How', 'long', 'does', 'shipping', 'take', '?', 'When', 'do', 'I', 'get', 'my', 'delivery', '?', 'Tell', 'me', 'a', 'joke', '!', 'Tell', 'me', 'something', 'funny', '!', 'Do', 'you', 'know', 'a', 'joke', '?']
tags: 
 ['greeting', 'goodbye', 'thanks', 'items', 'payments', 'delivery', 'funny']
xy: 
 [(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Ghaith\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## 2- prepare dataset:

In [2]:
ignore_words = ['?','!','.',',',':']
all_words = [Utils.stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print('all words: \n',all_words)
print('tags: \n',tags)


all words: 
 ["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
tags: 
 ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
import numpy as np
X_train = []
Y_train = []
for (pattern_sentence,tag) in xy :
    bag = Utils.bag_of_word(pattern_sentence,all_words)
    X_train.append(bag)
    target = tags.index(tag)
    Y_train.append(target)

X_train = np.array(X_train, dtype=np.float32)
Y_train = np.array(Y_train, dtype=np.int64)  


class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = Y_train
    
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.n_samples




##### define hyperparameters


In [4]:
batch_size = 8 
hidden_size = 8
learning_rate = 0.001
num_epochs = 1000
output_size = len(tags)
input_size = len(X_train[0])
dataset = ChatDataset()
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=1)


## 3- train model

In [5]:
from model import NeuralNet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)


NeuralNet(
  (l1): Linear(in_features=10, out_features=20, bias=True)
  (l2): Linear(in_features=20, out_features=20, bias=True)
  (l3): Linear(in_features=20, out_features=5, bias=True)
  (relu): ReLU()
)


### loss and optimizer

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


### training loop

In [7]:
for epoch in range(num_epochs):
    for i, (words, targets) in enumerate(train_loader):
        words = words.to(device)
        targets = targets.to(device)
        
        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Finished training, final loss =', loss.item())

RuntimeError: DataLoader worker (pid(s) 48376) exited unexpectedly